# maps_us_data.ipynb

Display interactive maps of the most recent COVID-19 statistics with and without normalization by population.

Inputs:
* `outputs/us_counties_clean.csv`: The contents of `data/us_counties.csv` after data cleaning by [clean_us_data.ipynb](./clean_us_data.ipynb)
* `outputs/us_counties_clean_meta.json`: Column type metadata for reading `data/us_counties_clean.csv` with `pd.read_csv()`
* [U.S. map in GeoJSON format, from Plotly](https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json)

**Note:** You can redirect these input files by setting the environment variable `COVID_OUTPUTS_DIR` to a replacement for the prefix `outputs` in the above paths.

In [1]:
# Initialization boilerplate
import os
import json
import pandas as pd
from urllib.request import urlopen

# Local file of utility functions
import util

# Allow environment variables to override data file locations.
_OUTPUTS_DIR = os.getenv("COVID_OUTPUTS_DIR", "outputs")
util.ensure_dir_exists(_OUTPUTS_DIR)  # create if necessary

In [2]:
# Read time series data from the binary file that clean_us_data.ipynb produces
dates_file = os.path.join(_OUTPUTS_DIR, "dates.feather")
cases_file = os.path.join(_OUTPUTS_DIR, "us_counties_clean.feather")
cases = pd.read_feather(cases_file).set_index("FIPS")
dates = pd.read_feather(dates_file)["date"].to_numpy()
cases.head()

,State,County,Population,Confirmed,Deaths,Recovered,Confirmed_Outlier,Deaths_Outlier,Recovered_Outlier,Confirmed_7_Days,Deaths_7_Days
FIPS,,,,,,,,,,,
1001,Alabama,Autauga,55869,"[ 0, 0, 0, 0, 0, 0, ...","[ 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",124,8
1003,Alabama,Baldwin,223234,"[ 0, 0, 0, 0, 0, 0, ...","[ 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",319,9
1005,Alabama,Barbour,24686,"[ 0, 0, 0, 0, 0, 0, ...","[ 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",20,3
1007,Alabama,Bibb,22394,"[ 0, 0, 0, 0, 0, 0, ...","[ 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",40,2
1009,Alabama,Blount,57826,"[ 0, 0, 0, 0, 0, 0, ...","[ 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",116,3


In [3]:
# Normalize the Confirmed and Deaths counts by population.
cases["Confirmed_per_100"] =  100.0 * cases["Confirmed"].array / cases["Population"].values.reshape(-1,1)
cases["Deaths_per_100"] = 100.0 * cases["Deaths"].array / cases["Population"].values.reshape(-1,1)

cases

,State,County,Population,Confirmed,Deaths,Recovered,Confirmed_Outlier,Deaths_Outlier,Recovered_Outlier,Confirmed_7_Days,Deaths_7_Days,Confirmed_per_100,Deaths_per_100
FIPS,,,,,,,,,,,,,
1001,Alabama,Autauga,55869,"[ 0, 0, 0, 0, 0, 0, ...","[ 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",124,8,"[ 0.0, 0.0,...","[ 0.0, 0.0,..."
1003,Alabama,Baldwin,223234,"[ 0, 0, 0, 0, 0, 0, ...","[ 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",319,9,"[ 0.0, 0.0,...","[ 0.0, 0.0,..."
1005,Alabama,Barbour,24686,"[ 0, 0, 0, 0, 0, 0, ...","[ 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",20,3,"[ 0.0, 0.0,...","[ 0.0, 0.0,..."
1007,Alabama,Bibb,22394,"[ 0, 0, 0, 0, 0, 0, ...","[ 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",40,2,"[ 0.0, 0.0,...","[ 0.0, 0.0,..."
1009,Alabama,Blount,57826,"[ 0, 0, 0, 0, 0, 0, ...","[ 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",116,3,"[ 0.0, 0.0,...","[ 0.0, 0.0,..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
56037,Wyoming,Sweetwater,42343,"[ 0, 0, 0, 0, 0, 0, ...","[ 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",120,3,"[ 0.0, 0.0,...","[ 0.0, 0.0,..."
56039,Wyoming,Teton,23464,"[ 0, 0, 0, 0, 0, 0, ...","[ 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",114,0,"[ 0.0, 0.0,...","[ 0.0, 0.0,..."
56041,Wyoming,Uinta,20226,"[ 0, 0, 0, 0, 0, 0, ...","[ 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",30,0,"[ 0.0, 0.0,...","[ 0.0, 0.0,..."


In [4]:
# Slice off the last element of each time series
latest_cases = cases[["State", "County", "Population"]].copy()
for col in ["Confirmed", "Confirmed_per_100", "Deaths", "Deaths_per_100"]:
    latest_cases[col] = cases[col].array._tensor[:,-1]
latest_cases

,State,County,Population,Confirmed,Confirmed_per_100,Deaths,Deaths_per_100
FIPS,,,,,,,
1001,Alabama,Autauga,55869,15431,27.619968,184,0.329342
1003,Alabama,Baldwin,223234,54763,24.531657,635,0.284455
1005,Alabama,Barbour,24686,5429,21.992222,92,0.372681
1007,Alabama,Bibb,22394,6354,28.373672,99,0.442083
1009,Alabama,Blount,57826,14672,25.372670,216,0.373534
...,...,...,...,...,...,...,...
56037,Wyoming,Sweetwater,42343,10937,25.829535,122,0.288123
56039,Wyoming,Teton,23464,9692,41.305830,15,0.063928
56041,Wyoming,Uinta,20226,5599,27.682191,36,0.177989


In [5]:
# Also show totals in the last week
cases_this_week = cases[["State", "County", "Population"]].copy()
cases_this_week["Confirmed"] = cases["Confirmed_7_Days"]
cases_this_week["Deaths"] = cases["Deaths_7_Days"]
cases_this_week["Confirmed_per_100"] = cases_this_week["Confirmed"] / cases_this_week["Population"]
cases_this_week["Deaths_per_100"] = cases_this_week["Deaths"] / cases_this_week["Population"]

cases_this_week

,State,County,Population,Confirmed,Deaths,Confirmed_per_100,Deaths_per_100
FIPS,,,,,,,
1001,Alabama,Autauga,55869,124,8,0.002219,0.000143
1003,Alabama,Baldwin,223234,319,9,0.001429,0.000040
1005,Alabama,Barbour,24686,20,3,0.000810,0.000122
1007,Alabama,Bibb,22394,40,2,0.001786,0.000089
1009,Alabama,Blount,57826,116,3,0.002006,0.000052
...,...,...,...,...,...,...,...
56037,Wyoming,Sweetwater,42343,120,3,0.002834,0.000071
56039,Wyoming,Teton,23464,114,0,0.004859,0.000000
56041,Wyoming,Uinta,20226,30,0,0.001483,0.000000


In [6]:
# Download a U.S. map in GEOJSON format
with urlopen('https://raw.githubusercontent.com/plotly/datasets/master/geojson-counties-fips.json') as response:
    counties = json.load(response)

In [7]:
# Common code to generate choropleth maps.
# NOTE: In order for this to work you need the JupyterLab extensions for Plotly:
#   > jupyter labextension install jupyterlab-plotly
# (env.sh will run the above command for you)

import plotly.express as px

def draw_map(df, col_name, label_str):
    # Each series may have NAs in different locations
    valid_data = df[~df[col_name].isna()]
    
    fig = px.choropleth(valid_data, geojson=counties, 
                        locations=["{:05d}".format(f) for f in valid_data.index],
                        color=col_name,
                        # See https://plotly.com/python/builtin-colorscales/
                        color_continuous_scale="viridis",
                        # Top of scale == 95th percentile
                        range_color=(0, valid_data[col_name].quantile(0.95)),
                        scope="usa",
                        labels={col_name: label_str},
                        hover_name=valid_data["County"],
                        title=label_str)
    fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})
    fig.show()
    


In [ ]:
# Draw a map of number of confirmed cases by county.
# Yellow == 95th percentile
draw_map(latest_cases, "Confirmed", "Confirmed Cases ")

In [ ]:
# Draw a map of number of confirmed cases in the past 7 days by county.
# Yellow == 95th percentile
draw_map(cases_this_week, "Confirmed", "Confirmed Cases this Week")

In [ ]:
# Draw a map of number of confirmed cases in the last week per 100 residents by county
# Yellow == 95th percentile
draw_map(cases_this_week, "Confirmed_per_100", "Confirmed per 100 this Week")

In [ ]:
# Draw a map of number of deaths per 100 residents by county
# Yellow == 95th percentile
draw_map(cases_this_week, "Deaths_per_100", "Deaths per 100 this Week")

In [12]:
# Draw a map of number of confirmed cases per 100 residents by county
# Yellow == 95th percentile
#draw_map(latest_cases, "Confirmed_per_100", "Confirmed per 100")

In [13]:
# Draw a map of number of deaths by county
# Yellow == 95th percentile
#draw_map(latest_cases, "Deaths", "Total Deaths  ")

In [14]:
# Draw a map of number of deaths per 100 residents by county
# Yellow == 95th percentile
#draw_map(latest_cases, "Deaths_per_100", "Deaths per 100")